<a href="https://colab.research.google.com/github/mandarbagwe7/Hands-On-Large-Language-Models-Practice/blob/main/chapter_02/Token_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import AutoModel, AutoTokenizer

In [ ]:
# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

In [ ]:
# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

In [5]:
# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

In [6]:
# Process the tokens
output = model(**tokens)[0]
output

tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

In [7]:
output.shape

torch.Size([1, 4, 384])

In [9]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
Hello
 world
[SEP]
